#Step 1:
Installing the required packages

In [ ]:
pip install yfinance pandas

#Step 2:
Writing the python scripts to download stock data

In [ ]:
#import libraries

import yfinance as yf
import pandas as pd
import glob
from datetime import datetime

In [ ]:


# List of stocks being tracked
tickers_info = {
    "AAPL": "Technology", #Apple Inc
    "MSFT": "Technology", #Microsoft Corporation
    "TSLA": "Automotive", #Tesla
    "AMZN": "E-commerce", #Amazon.com Inc
    "GOOG": "Technology", #Alphabet Inc (Google)
    "META": "Technology/Social Media", #Meta Platforms Inc (Facebook)
    "NVDA": "Semiconductors",  #NVIDIA Corporation
    "JPM": "Banking",          #JPMorgan Chase & Co
    "XOM": "Energy",           #Exxon Mobil Corporation
    "BRK-B": "Conglomerate"    #Berkshire Hathaway Inc. (Class B)
}


#Define date range

start_date = "2020-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

#Downloading data

data = yf.download(list(tickers_info.keys()), start=start_date, end=end_date, group_by='ticker', auto_adjust=False)

#step 3: processing data

all_data = [] #created an empty list to house the processed data

for ticker, sector in tickers_info.items():
    df = data[ticker].copy()  #gets only ticker data
    df['Ticker'] = ticker
    df['Sector'] = sector
    df['Daily Return'] = df['Adj Close'].pct_change()
    df['Cumulative Return'] = (1 + df['Daily Return']).cumprod() - 1
    df['MA50'] = df['Adj Close'].rolling(window=50).mean()
    df['MA200'] = df['Adj Close'].rolling(window=200).mean()
    df = df.reset_index()
    all_data.append(df)

# Step 4: Combining all stocks
final_df = pd.concat(all_data)

# STEP 5: Save to CSV
final_df.to_csv("stock_data.csv", index=False)
print("✅ Stock data with sectors saved to 'stock_data.csv'")


[*********************100%***********************]  10 of 10 completed


✅ Stock data with sectors saved to 'stock_data.csv'


#Nigeria Stocks
To download Nigeria stock data, I will be making use of investing.com

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# Change path to where your folder is in Google Drive
import glob
nigerian_files = glob.glob("/content/drive/MyDrive/nigerian_stocks/*.csv")

Mounted at /content/drive


In [ ]:
#dictionary of Nigerian stocks and sectors
nigerian_stocks = {
    "GTCO": "Banking",  #GTBank
    "ZENITHBANK": "Banking",   #Zenith Bank
    "MTNN": "Telecoms",        #MTN Nigeria
    "DANGCEM": "Industrial",   #Dangote Cement
    "AIRTELAFRI": "Telecoms"   #Airtel Africa
}

nigerian_data_list = []

# Loop through all CSV files in the "nigerian_stocks" folder
for file in glob.glob("/content/drive/MyDrive/nigerian_stocks/*.csv"):
    ticker = file.split("/")[-1].replace(".csv", "").upper()  # Get ticker from filename

    if ticker in nigerian_stocks:
        df = pd.read_csv(file)

 # Rename columns if needed (Investing.com format)
        df.rename(columns={
            'Price': 'Adj Close',
            'Open': 'Open',
            'High': 'High',
            'Low': 'Low',
            'Vol.': 'Volume'
        }, inplace=True, errors='ignore')

        # Convert Date to datetime format
        df['Date'] = pd.to_datetime(df['Date'])

        # Sort by date
        df = df.sort_values('Date')

  # 🔹 CLEAN numeric columns: remove commas and convert to float
        for col in ['Adj Close', 'Open', 'High', 'Low', 'Volume']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

        # Add calculated columns
        df['Ticker'] = ticker
        df['Sector'] = nigerian_stocks[ticker]
        df['Daily Return'] = df['Adj Close'].pct_change()
        df['Cumulative Return'] = (1 + df['Daily Return']).cumprod() - 1
        df['MA50'] = df['Adj Close'].rolling(window=50).mean()
        df['MA200'] = df['Adj Close'].rolling(window=200).mean()
        df['Country'] = "Nigeria"

        nigerian_data_list.append(df)

# Combine all Nigerian stocks into one DataFrame
nigeria_df = pd.concat(nigerian_data_list, ignore_index=True)

print(f"✅ Nigerian stock data ready! Shape: {nigeria_df.shape}")

✅ Nigerian stock data ready! Shape: (6926, 14)


Now, to combine the international data with the Nigerian data

In [ ]:
combined_df = pd.concat([final_df, nigeria_df], ignore_index=True)
combined_df.to_csv("combined_stock_data.csv", index=False)
print("✅ Final dataset saved as 'combined_stock_data.csv'")


✅ Final dataset saved as 'combined_stock_data.csv'


To clean and validate combined data
- check for missing values
- remove outliers
- ensure date column is in proper date time format

In [ ]:
df = pd.read_csv("combined_stock_data.csv")
df['Date'] = pd.to_datetime(df['Date'])
print(df.info())
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21026 entries, 0 to 21025
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               21026 non-null  datetime64[ns]
 1   Open               21026 non-null  float64       
 2   High               21026 non-null  float64       
 3   Low                21026 non-null  float64       
 4   Close              14100 non-null  float64       
 5   Adj Close          21026 non-null  float64       
 6   Volume             14100 non-null  float64       
 7   Ticker             21026 non-null  object        
 8   Sector             21026 non-null  object        
 9   Daily Return       21011 non-null  float64       
 10  Cumulative Return  21011 non-null  float64       
 11  MA50               20291 non-null  float64       
 12  MA200              18041 non-null  float64       
 13  Change %           6926 non-null   object        
 14  Countr

In [ ]:
#to download final file to local computer
from google.colab import files
files.download("combined_stock_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>